In [8]:
import random
import operator

# --- Definición de funciones y terminales ---
FUNCTIONS = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': lambda a, b: a / b if b != 0 else 1
}
TERMINALS = ["y", 1, 2, 3, 5]  # sin 10 ni números grandes

# --- Generación de un árbol aleatorio ---
def generate_tree(depth=3):
    if depth == 0 or (random.random() < 0.3):
        return random.choice(TERMINALS)
    func = random.choice(list(FUNCTIONS.keys()))
    return [func, generate_tree(depth-1), generate_tree(depth-1)]

# --- Evaluación del árbol dado un valor de y ---
def eval_tree(tree, y):
    if isinstance(tree, (int, float)):
        return tree
    if tree == "y":
        return y
    func = FUNCTIONS[tree[0]]
    return func(eval_tree(tree[1], y), eval_tree(tree[2], y))

# --- Simulación del sistema dinámico ---
def simulate(tree, y0=0, steps=10, ref=37):
    y = y0
    for _ in range(steps):
        u = eval_tree(tree, y)
        y = y + u
    return y

# --- Función de aptitud ---
def fitness(tree, ref=37):
    y_final = simulate(tree)
    return 1 / (1 + abs(y_final - ref))

# --- Mutación ---
def mutate(tree, depth=3):
    if random.random() < 0.2:
        return generate_tree(depth)
    if isinstance(tree, list):
        return [tree[0], mutate(tree[1], depth-1), mutate(tree[2], depth-1)]
    return tree

# --- Cruce ---
def crossover(t1, t2):
    if isinstance(t1, list) and isinstance(t2, list) and random.random() < 0.7:
        return [t1[0], crossover(t1[1], t2[1]), crossover(t1[2], t2[2])]
    return t1 if random.random() < 0.5 else t2

# --- Algoritmo Genético ---
def genetic_programming(generations=15, pop_size=12):
    population = [generate_tree(3) for _ in range(pop_size)]
    for g in range(generations):
        scored = [(fitness(ind), ind) for ind in population]
        scored.sort(reverse=True, key=lambda x: x[0])
        print(f"Generación {g+1}: Mejor aptitud = {scored[0][0]:.4f}")
        # Nueva población: elitismo + cruces/mutaciones
        new_pop = [scored[0][1]]  # elitismo
        while len(new_pop) < pop_size:
            _, p1 = random.choice(scored[:5])
            _, p2 = random.choice(scored[:5])
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
    return scored[0][1], scored[0][0]

# --- Ejecutar ---
best_tree, best_fit = genetic_programming()
print("\nMejor controlador encontrado:")
print(best_tree)
print(f"Aptitud: {best_fit:.4f}, y_final = {simulate(best_tree):.4f}")


Generación 1: Mejor aptitud = 0.0833
Generación 2: Mejor aptitud = 0.5000
Generación 3: Mejor aptitud = 0.5000
Generación 4: Mejor aptitud = 0.5000
Generación 5: Mejor aptitud = 0.5000
Generación 6: Mejor aptitud = 0.5000
Generación 7: Mejor aptitud = 0.5000
Generación 8: Mejor aptitud = 0.5000
Generación 9: Mejor aptitud = 0.6860
Generación 10: Mejor aptitud = 0.6860
Generación 11: Mejor aptitud = 0.6860
Generación 12: Mejor aptitud = 0.6860
Generación 13: Mejor aptitud = 0.6860
Generación 14: Mejor aptitud = 0.6860
Generación 15: Mejor aptitud = 0.6860

Mejor controlador encontrado:
['*', 3, ['+', 1, ['/', 2, 'y']]]
Aptitud: 0.6860, y_final = 36.5422


In [15]:
import random
import operator
import math

# --- Definición de funciones y terminales ---
FUNCTIONS = {
    '+': operator.add,
    '-': operator.sub,
    '*': operator.mul,
    '/': lambda a, b: a / b if b != 0 else 1
}
TERMINALS = ["y", 1, 2, 3, 5]

# --- Generación de un árbol aleatorio ---
def generate_tree(depth=3):
    if depth == 0 or (random.random() < 0.3):
        return random.choice(TERMINALS)
    func = random.choice(list(FUNCTIONS.keys()))
    return [func, generate_tree(depth-1), generate_tree(depth-1)]

# --- Evaluación del árbol ---
def eval_tree(tree, y):
    if isinstance(tree, (int, float)):
        return tree
    if tree == "y":
        return y
    func = FUNCTIONS[tree[0]]
    return func(eval_tree(tree[1], y), eval_tree(tree[2], y))

# --- Simulación del sistema dinámico ---
def simulate(tree, y0=0, steps=10, ref=37):
    y = y0
    for _ in range(steps):
        u = eval_tree(tree, y)
        y = y + u
        # protección contra valores enormes
        if abs(y) > 1e6:
            return float("inf")
    return y

# --- Función de aptitud protegida ---
def fitness(tree, ref=37):
    y_final = simulate(tree)
    if y_final == float("inf") or math.isnan(y_final):
        return 0.0
    error = abs(y_final - ref)
    error = min(error, 1e6)   # recorte para evitar overflow
    return 1 / (1 + error/10)

# --- Mutación ---
def mutate(tree, depth=3):
    if random.random() < 0.4:
        return generate_tree(depth)
    if isinstance(tree, list):
        return [tree[0], mutate(tree[1], depth-1), mutate(tree[2], depth-1)]
    return tree

# --- Cruce ---
def crossover(t1, t2):
    if isinstance(t1, list) and isinstance(t2, list) and random.random() < 0.7:
        return [t1[0], crossover(t1[1], t2[1]), crossover(t1[2], t2[2])]
    return t1 if random.random() < 0.5 else t2

# --- Algoritmo Genético ---
def genetic_programming(generations=100, pop_size=50):
    population = [generate_tree(3) for _ in range(pop_size)]
    best_scores = []
    for g in range(generations):
        scored = [(fitness(ind), ind) for ind in population]
        scored.sort(reverse=True, key=lambda x: x[0])
        best_scores.append(scored[0][0])
        print(f"Generación {g+1}: Mejor aptitud = {scored[0][0]:.4f}")
        # Nueva población: elitismo + cruces/mutaciones
        new_pop = [scored[0][1]]
        while len(new_pop) < pop_size:
            _, p1 = random.choice(scored[:10])
            _, p2 = random.choice(scored[:10])
            child = crossover(p1, p2)
            child = mutate(child)
            new_pop.append(child)
        population = new_pop
    return scored[0][1], scored[0][0], best_scores

# --- Ejecutar ---
best_tree, best_fit, history = genetic_programming()

print("\nMejor controlador encontrado:")
print(best_tree)
print(f"Aptitud: {best_fit:.4f}, y_final = {simulate(best_tree):.4f}")


Generación 1: Mejor aptitud = 0.7692
Generación 2: Mejor aptitud = 0.7692
Generación 3: Mejor aptitud = 0.7692
Generación 4: Mejor aptitud = 0.7692
Generación 5: Mejor aptitud = 0.7692
Generación 6: Mejor aptitud = 0.7692
Generación 7: Mejor aptitud = 0.7692
Generación 8: Mejor aptitud = 0.7692
Generación 9: Mejor aptitud = 0.7692
Generación 10: Mejor aptitud = 0.9524
Generación 11: Mejor aptitud = 0.9677
Generación 12: Mejor aptitud = 0.9677
Generación 13: Mejor aptitud = 0.9677
Generación 14: Mejor aptitud = 0.9677
Generación 15: Mejor aptitud = 0.9677
Generación 16: Mejor aptitud = 0.9677
Generación 17: Mejor aptitud = 0.9677
Generación 18: Mejor aptitud = 0.9677
Generación 19: Mejor aptitud = 0.9677
Generación 20: Mejor aptitud = 0.9677
Generación 21: Mejor aptitud = 0.9677
Generación 22: Mejor aptitud = 0.9677
Generación 23: Mejor aptitud = 0.9677
Generación 24: Mejor aptitud = 0.9677
Generación 25: Mejor aptitud = 0.9677
Generación 26: Mejor aptitud = 0.9677
Generación 27: Mejor 